In [1]:
import os
import pandas as pd
import glob

# Directory containing the CSV files
directory = r"D:\HuaiReachCatchment20192023daily\timeseries\flow"

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(directory, "*.csv"))

# Initialize counters
total_files = len(csv_files)
files_with_negatives = 0
total_negatives = 0

# Check each file for negative Qout values
print(f"Checking {total_files} CSV files for negative Qout values...")
print("-" * 70)

for file_path in csv_files:
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if 'Qout' column exists
        if 'Qout' in df.columns:
            # Find negative values in Qout
            negative_values = df[df['Qout'] < 0]
            neg_count = len(negative_values)
            
            # Report if negative values found
            if neg_count > 0:
                files_with_negatives += 1
                total_negatives += neg_count
                print(f"File: {os.path.basename(file_path)}")
                print(f"  - Contains {neg_count} negative Qout values")
                print(f"  - Negative values: {negative_values['Qout'].tolist()}")
                print(f"  - At indices: {negative_values.index.tolist()}")
                print("-" * 70)
        else:
            print(f"Warning: No 'Qout' column found in {os.path.basename(file_path)}")
            
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {e}")

# Summary report
print("\nSummary:")
print(f"Total files checked: {total_files}")
print(f"Files with negative Qout values: {files_with_negatives}")
print(f"Total negative Qout values found: {total_negatives}")

if files_with_negatives == 0:
    print("No negative Qout values found in any file.")

Checking 7825 CSV files for negative Qout values...
----------------------------------------------------------------------
File: 43018325_flow.csv
  - Contains 3 negative Qout values
  - Negative values: [-2.088505268096924, -0.0481823571026325, -0.0761210471391677]
  - At indices: [222, 1369, 1370]
----------------------------------------------------------------------
File: 43023349_flow.csv
  - Contains 1 negative Qout values
  - Negative values: [-0.030213737860322]
  - At indices: [939]
----------------------------------------------------------------------
File: 43025192_flow.csv
  - Contains 1 negative Qout values
  - Negative values: [-0.1593489050865173]
  - At indices: [934]
----------------------------------------------------------------------
File: 43025193_flow.csv
  - Contains 2 negative Qout values
  - Negative values: [-0.454703539609909, -0.3169443905353546]
  - At indices: [933, 934]
----------------------------------------------------------------------
File: 43025197_f

In [2]:
import xarray as xr
import os

# Path to the NetCDF file
nc_file = r"E:\\Research\\Research_Data\\GRADES-hydro\\output_pfaf_04_202309.nc"

# Check if the file exists
if not os.path.exists(nc_file):
    print(f"Error: File not found at {nc_file}")
else:
    try:
        # Open the NetCDF file
        ds = xr.open_dataset(nc_file)
        
        # Print information about the dataset
        print("Dataset information:")
        print("-" * 50)
        print(ds.info())
        
        # List all variables in the file
        print("\nVariables in the NetCDF file:")
        print("-" * 50)
        for var_name, var in ds.variables.items():
            print(f"Variable: {var_name}")
            print(f"  Dimensions: {var.dims}")
            print(f"  Shape: {var.shape}")
            print(f"  Data type: {var.dtype}")
            if hasattr(var, 'units'):
                print(f"  Units: {var.units}")
            if hasattr(var, 'long_name'):
                print(f"  Description: {var.long_name}")
            print("-" * 30)
        
        # Close the dataset
        ds.close()
        
    except Exception as e:
        print(f"Error reading NetCDF file: {e}")

Dataset information:
--------------------------------------------------
xarray.Dataset {
dimensions:
	time = 30 ;
	rivid = 464949 ;
	nerr = 3 ;
	nv = 2 ;

variables:
	float32 Qout(time, rivid) ;
		Qout:long_name = average river water discharge downstream of each river reach ;
		Qout:units = m3 s-1 ;
		Qout:grid_mapping = crs ;
		Qout:cell_methods = time: mean ;
	float32 Qout_err(nerr, rivid) ;
		Qout_err:long_name = average river water discharge uncertainty downstream of each river reach ;
		Qout_err:units = m3 s-1 ;
		Qout_err:grid_mapping = crs ;
		Qout_err:cell_methods = time: mean ;
	int32 rivid(rivid) ;
		rivid:long_name = unique identifier for each river reach ;
		rivid:units = 1 ;
		rivid:cf_role = timeseries_id ;
	datetime64[ns] time(time) ;
		time:standard_name = time ;
		time:long_name = time ;
		time:axis = T ;
		time:bounds = time_bnds ;
	datetime64[ns] time_bnds(time, nv) ;
	float64 lon(rivid) ;
		lon:standard_name = longitude ;
		lon:long_name = longitude of a point relat

In [4]:
import xarray as xr
import numpy as np
import os


# Check if the file exists
if not os.path.exists(nc_file):
    print(f"Error: File not found at {nc_file}")
else:
    try:
        # Open the NetCDF file
        print(f"Opening NetCDF file: {nc_file}")
        ds = xr.open_dataset(nc_file)
        
        # List all variables in the file
        print(f"\nVariables in the file: {list(ds.variables)}")
        
        # Check if Qout variable exists
        if 'Qout' in ds:
            qout = ds['Qout']
            
            # Get basic information about Qout
            print(f"\nQout variable information:")
            print(f"  Dimensions: {qout.dims}")
            print(f"  Shape: {qout.shape}")
            if hasattr(qout, 'units'):
                print(f"  Units: {qout.units}")
                
            # Check for negative values
            negative_values = qout.values < 0
            num_negative = np.sum(negative_values)
            
            if num_negative > 0:
                print(f"\nFound {num_negative} negative values in Qout")
                
                # Get statistics about negative values
                min_value = float(qout.values.min())
                percent_negative = 100 * num_negative / qout.size
                
                print(f"  Minimum value: {min_value}")
                print(f"  Percentage of negative values: {percent_negative:.6f}%")
                
                # Get indices of some negative values (limit to first 10)
                neg_indices = np.where(negative_values)
                print(f"\nSample of negative values (first 10 occurrences):")
                for i in range(min(10, num_negative)):
                    idx = tuple(index[i] for index in neg_indices)
                    value = float(qout.values[idx])
                    print(f"  Index: {idx}, Value: {value}")
            else:
                print("\nNo negative values found in Qout")
                print(f"Minimum value: {float(qout.values.min())}")
        else:
            print("\nQout variable not found in the NetCDF file")
            
        # Close the dataset
        ds.close()
        
    except Exception as e:
        print(f"Error analyzing NetCDF file: {e}")

Opening NetCDF file: E:\\Research\\Research_Data\\GRADES-hydro\\output_pfaf_04_202309.nc

Variables in the file: ['Qout', 'Qout_err', 'rivid', 'time', 'time_bnds', 'lon', 'lat', 'crs']

Qout variable information:
  Dimensions: ('time', 'rivid')
  Shape: (30, 464949)
  Units: m3 s-1

Found 1956 negative values in Qout
  Minimum value: -2.659370183944702
  Percentage of negative values: 0.014023%

Sample of negative values (first 10 occurrences):
  Index: (0, 30425), Value: -5.4163974709808826e-05
  Index: (0, 38308), Value: -0.00046483014011755586
  Index: (0, 41272), Value: -0.00019939517369493842
  Index: (0, 41627), Value: -0.0004280731955077499
  Index: (0, 49602), Value: -0.0003321358235552907
  Index: (0, 49617), Value: -0.00030398921808227897
  Index: (0, 50780), Value: -0.00024290880537591875
  Index: (0, 77841), Value: -4.73494914388084e-08
  Index: (0, 77844), Value: -5.738900199503405e-08
  Index: (0, 77885), Value: -0.000277240906143561
